In [1]:
import time

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from matplotlib.image import imread
from matplotlib.patches import Rectangle
from matplotlib.colors import TwoSlopeNorm

from tomotok.core.phantoms import gauss_iso
from tomotok.core.geometry import RegularGrid, sparse_line_3d, generate_los
from tomotok.core.derivative import derivative_matrix
from tomotok.core.inversions import GevFastAlgebraic, Mfr, SimpleBob, SvdFastAlgebraic

In [2]:
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 6
rcParams['figure.dpi'] = 200
rcParams['image.origin'] = 'lower'
rcParams['image.cmap'] = 'RdBu'
rcParams['lines.linewidth'] = 1

In [3]:
svd = SvdFastAlgebraic()
gev = GevFastAlgebraic()
mfr = Mfr()

# Linear Detectors
 - sets up a system of linear array detectors
 - computes geometry matrix
 - creates phantom emissivity 
 - computes synthetic signals 
 - performs inversions using SVD and GEV variants of Fast Algebraic version of LAME algorithm
 - performs inversion using MFR 

In [4]:
# grit limits
rlim = (.3, .7)
zlim = (-.4, .4)

# create inversion grid with desired node size
# grid = RegularGrid(10, 20, rlim, zlim)  # 4x4cm
# grid = RegularGrid(20, 40, rlim, zlim)  # 2x2cm, used for paper figures
grid = RegularGrid(40, 80, rlim, zlim)  # 1x1cm
# grid = RegularGrid(80, 160, rlim, zlim)  # .5x.5cm

In [5]:
num = 20  # detectors per array
s1, e1 = generate_los(num=(num, 1), fov=(70, 0), pinhole=(1, 0, 0), axis=(-1, 0, 0))  # horizontal
s2, e2 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.5, 0, .7), axis=(.01, 0, -1), elong=1.5)  # top
s3, e3 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.9, 0, -.5), axis=(-1, 0, 1), elong=1.5)  # angled bottom
s4, e4 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.9, 0, .5), axis=(-1, 0, -1), elong=1.5)  # angled top

# combine line of sights coordinates of arrays into one variable
ss = []
es = []
ss.append(s1)
es.append(e1)
ss.append(s2)
es.append(e2)
ss.append(s3)
es.append(e3)
es.append(e4)
ss.append(s4)
s = np.concatenate(ss, 0)
e = np.concatenate(es, 0)

In [6]:
chnls = np.arange(s.shape[0])
rch = np.zeros((s.shape[0], 2))
zch = np.zeros((s.shape[0], 2))
rch[:, 0] = s[:, 0]
rch[:, 1] = e[:, 0]
zch[:, 0] = s[:, 2]
zch[:, 1] = e[:, 2]

In [ ]:
# display line of sights and the extent of the grid
losf = plt.figure(
  # figsize=(2., 2.), 
  figsize=(1.5, 1.5),
  dpi=200
)
ax = plt.axes()
ax.set_aspect(1)
ax.set_xlim(0, 1)
ax.set_ylim(-0.5, 0.7)

ax.set_xlabel('R [-]')
ax.set_ylabel('z [-]')

ax.plot(rch.T, zch.T, 'k', lw=0.5, alpha=0.5)
rct = Rectangle((grid.rmin, grid.zmin), grid.rmax - grid.rmin, grid.zmax - grid.zmin)
ax.add_patch(rct, )

# losf.savefig('los.png', bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
gmat = sparse_line_3d(rch, zch, grid, rmin=.2)
dgmat = gmat.toarray()

In [9]:
phantom = gauss_iso(grid.nr, grid.nz, w=.2) * 100

In [ ]:
ad1 = plt.figure(
    # figsize=(1., 2.), 
    figsize=(1.5, 1.5),
    dpi=200
)
ad1ax = ad1.add_subplot(111)
ad1ax.set_aspect(1)
ad1ax.set_xlabel('R [-]')
ad1ax.set_ylabel('z [-]')

ad1img = ad1ax.imshow(phantom, extent=grid.extent, origin='lower', cmap='Blues', vmax=100)

ad1cax = ad1ax.inset_axes(bounds=[1.1, 0., .05, 1])

ad1.colorbar(ad1img, cax=ad1cax, ax=ad1ax, label='Emissivity [-]')

# ad1.savefig('phantom1.png', bbox_inches='tight', pad_inches=0)
plt.show()

In [11]:
sig = gmat.dot(phantom.flatten())

# add noise
# ampl_noise = sig.max() * .02
# sig += np.random.normal(0, ampl_noise, sig.size)

data = sig.reshape(1, -1)  # data should have shape (#time slices, #channels/pixels)

# expected error in data
errors = .001

In [12]:
# derivative matrices for the inversion
derivs = [
    [derivative_matrix(grid, 'right'), derivative_matrix(grid, 'top')],
    [derivative_matrix(grid, 'top'), derivative_matrix(grid, 'left')],
]

In [ ]:
ela = time.time()
sout = svd(data, dgmat, derivatives=derivs, errors=errors, method='logmean')
ela = time.time() - ela
print('svd', ela, 's')

In [ ]:
ela = time.time()
gout = gev(data, dgmat, derivs, errors, method='logmean')
ela = time.time() - ela
print('gev', ela, 's')

In [ ]:
mout = mfr(data, gmat, derivs, errors)

# Matrix Camera
- sets up a system based on tangentially viewing matrix camera
- computes geometry matrix
- creates phantom emissivity 
- computes synthetic image
- performs inversions using MFR and BOB

In [16]:
# matrix camera definition
# resolution = (40, 40)  # old camera resolution
resolution = (80, 80)  # new camera resolution
fov = (60, 60)
pinhole_position = (0.8, 0.1, 0.2)
camera_axis = (-1, -0.2, 0.25,)

start, end = generate_los(
    pinhole=pinhole_position,
    axis=camera_axis,
    num=resolution, 
    fov=fov, 
    elong=3,
)

nch = start.shape[0]
xc = np.zeros((nch, 2))
yc = np.zeros((nch, 2))
zc = np.zeros((nch, 2))
xc[:, 0] = start[:, 0]
xc[:, 1] = end[:, 0]
yc[:, 0] = start[:, 1]
yc[:, 1] = end[:, 1]
zc[:, 0] = start[:, 2]
zc[:, 1] = end[:, 2]

In [ ]:
grid2 = RegularGrid(30, 40, (.2, .8), (-.4, .4))
gmat2 = sparse_line_3d(xc, yc, grid2, zc, rmin=.2)

grid_column = RegularGrid(1, 1, (0, 0.2, ), (-.4, .4))
gmat_column = sparse_line_3d(xc, yc, grid_column, zc)
image_column = gmat_column.dot(np.array([1]))

In [18]:
phantom2 = gauss_iso(grid2.nr, grid2.nz, cen=.5) * 100
image = gmat2.dot(phantom2.reshape(-1, 1))
derivs2 = [
    [derivative_matrix(grid2, 'right'), derivative_matrix(grid2, 'top')],
    [derivative_matrix(grid2, 'left'), derivative_matrix(grid2, 'bottom')],
]

In [ ]:
bob = SimpleBob()
ela2 = time.time()
bob.decompose(gmat2)
ela2 = time.time() - ela2

In [ ]:
bout = bob(image, gmat2)
mout2 = mfr(
    data=image.reshape(1, -1), 
    gmat=gmat2, 
    errors=np.ones((1, image.size))*1e-5,
    derivatives=derivs2,
    bounds=(-15,0),
)

# Paper figures

## Figure 2

In [ ]:
# layout of linear array detectors + a node projection in compass tokamak from CALCAM + artificial image obtained from the matrix camera

lna = plt.figure(figsize=(6, 1.5))
lnaax = lna.subplots(1, 3)


lnaax[0].set_aspect(1)
lnaax[0].set_xlim(0, 1)
lnaax[0].set_xticks((0, grid.rmin, grid.rmax, 1))
lnaax[0].set_ylim(-.5, .7)
lnaax[0].set_title('Linear Layout')
lnaax[0].set_xlabel('R [-]')
lnaax[0].set_ylabel('z [-]')

lnaax[2].set_axis_off()
lnaax[2].set_title('Artificial Image')

lnaax[0].plot(rch.T, zch.T, 'k', lw=0.5, alpha=0.5)
rct = Rectangle((grid.rmin, grid.zmin), grid.rmax - grid.rmin, grid.zmax - grid.zmin)
lnaax[0].add_patch(rct, )

# The node projections with a wireframe of COMPASS vessel was calculated separately in CALCAM
# It can not be easily reproduced here, so the image is loaded if available
# Otherwise a simple node projection is drawn
try:
    lnaax[1].set_title('Node Projection')
    node = imread('node_image_plane.png')
    lnaax[1].imshow(node, origin='upper')
    lnaax[1].set_axis_off()
except FileNotFoundError:
    from matplotlib.colors import LinearSegmentedColormap
    cmap = LinearSegmentedColormap.from_list('foo', [(0, 'white'), (1, 'C1')])
    tmp = np.zeros_like(phantom2)
    tmp[18, 17] = 100
    replacement = gmat2 @ tmp.flatten()
    replacement = replacement.reshape(resolution).T > 0
    lnaax[1].imshow(replacement, cmap=cmap, origin='lower')
    lnaax[1].set_axis_off()

ncimg = lnaax[2].imshow(image.reshape(resolution).T, cmap='Blues', origin='lower')
nccax = lnaax[2].inset_axes(bounds=[1.1, 0., .05, 1])
lna.colorbar(ncimg, cax=nccax, label='Signal [-]')

lnaax[2].text(3.2/4*resolution[0], 1/4*resolution[1], 'Central column', rotation='vertical')
lnaax[2].contour(image_column.reshape(resolution).T, levels=[0], colors='k', linewidths=0.5)

# lna.savefig('setups.png', bbox_inches='tight', pad_inches=0)

plt.show()


## Figure 3

In [ ]:
# Results of MFR + LAME compared with phantom

resf1 = plt.figure(figsize=(4, 1.5), dpi=200)
resax = resf1.subplots(1, 4, sharey=True, gridspec_kw={'wspace': .05})

datas = [phantom, mout[0], sout[0], gout[0]]
titles = ['Phantom', 'MFR', 'SVD', 'GEV']

vmin = min([d.min() for d in datas])
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=100)

resax[0].set_ylabel('z [-]')
for i in range(4):
    resax[i].set_title(titles[i])
    resax[i].set_xlabel('R [-]')
    # resax[i].set_ylabel('z [-]')
    resax[i].set_xticks((grid.rmin, grid.r_center.mean(), grid.rmax))

    resimg = resax[i].imshow(
        datas[i].reshape(grid.shape), 
        extent=grid.extent, 
        norm=norm,
        # cmap='cividis',
    )

rescax = resax[-1].inset_axes(bounds=[1.2, 0., 0.05, 1])

resfcbar = resf1.colorbar(
    resimg, cax=rescax, label='Emissivity [-]', 
    ticks=[-10, -5, 0, 50, 100],
    spacing='proportional',
    )

# resf1.savefig('res1.png', bbox_inches='tight', pad_inches=0)

plt.show()

## Figure 4

In [ ]:
# Horizontal and vertical cuts through the phantom and the reconstructions using linear system

col = grid.nr // 2 - 1
row = grid.nz // 2 

cutf = plt.figure(figsize=(6,1.5))
cutax = cutf.subplots(1, 3)

cutax[0].set_title('Phantom')
cutax[0].set_xlabel('R [-]')
cutax[0].set_ylabel('z [-]')
cutax[0].set_xticks((grid.rmin, grid.r_center.mean(), grid.rmax))
cutax[0].imshow(phantom, cmap='Blues', extent=grid.extent)
cutax[0].axhline(grid.z_border[row], lw=1, color='k', ls='--')
cutax[0].axvline(grid.r_border[col+1], lw=1, color='k', ls='--')

cutax[1].set_title('Horizontal')
cutax[1].set_xlabel('R [-]')
cutax[1].set_ylabel('Emissivity [-]')
lnp, = cutax[1].plot(grid.r_center, phantom[row], label='Phantom', lw=1)
lnm, = cutax[1].plot(grid.r_center, mout[0].reshape(grid.shape)[row], label='MFR', lw=1)
lng, = cutax[1].plot(grid.r_center, gout[0].reshape(grid.shape)[row], label='GEV', lw=1)
lns, = cutax[1].plot(grid.r_center, sout[0].reshape(grid.shape)[row], label='SVD', lw=1)

cutax[2].set_title('Vertical')
cutax[2].yaxis.tick_right()
cutax[2].set_ylabel('z [-]')
cutax[2].set_xlabel('Emissivity [-]')
cutax[2].yaxis.set_label_position('right')
cutax[2].plot(phantom[:, col], grid.z_center, label='Phantom', lw=1)
cutax[2].plot(mout[0].reshape(grid.shape)[:, col], grid.z_center, label='MFR', lw=1)
cutax[2].plot(gout[0].reshape(grid.shape)[:, col], grid.z_center, label='GEV', lw=1)
cutax[2].plot(sout[0].reshape(grid.shape)[:, col], grid.z_center, label='SVD', lw=1)

cutf.legend(handles=[lnp, lnm, lng, lns], bbox_to_anchor=[0.58, .5], loc='center left', ncol=1)

# cutf.savefig(+'cuts.png', bbox_inches='tight', pad_inches=0)
plt.show()

## Figure 5

In [ ]:
# Results of matrix camera setup of MFR and BOB

resf2 = plt.figure(figsize=(5, 1.5), dpi=200)
resax2 = resf2.subplots(1, 3)

r2titles = ['Phantom', 'MFR', 'BOB']

norm2 = TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=100)
kw2 = dict(extent=grid2.extent, norm=norm2)
kwcbar2 = dict(label='Emissivity [-]', ticks=[-0.4, -0.2, 0, 50, 100])

for i, ax in enumerate(resax2):
    ax.set_xlabel('R [-]')
    ax.set_ylabel('z [-]')
    ax.set_title(r2titles[i])
    ax.set_xticks((grid2.rmin, grid2.r_center.mean(), grid2.rmax))

resax2[0].imshow(phantom2, **kw2)
resax2[1].imshow(mout2[0].reshape(grid2.shape), **kw2)
res2img = resax2[2].imshow(bout.reshape(grid2.shape), **kw2)

res2cax = resax2[2].inset_axes(bounds=[1.3, 0, 0.05, 1])
resf2cbar = resf2.colorbar(res2img, cax=res2cax, **kwcbar2)

# resf2.savefig('res2.png', bbox_inches='tight', pad_inches=0)

plt.show()

## Unused

In [ ]:
# Artificial image and results merged in one figure

merged = plt.figure(figsize=(6, 1.5))
meax = merged.subplots(1, 5)
ofs=2

mtitles = ['Phantom', 'MFR', 'BOB']

for i, ax in enumerate(meax[ofs:]):
    ax.set_xlabel('R [-]')
    ax.set_title(mtitles[i])

meax[0].set_axis_off()
meax[1].set_axis_off()

meax[0+ofs].set_ylabel('z [-]')
meax[1+ofs].set_yticklabels(())
meax[2+ofs].set_yticklabels(())

meax[0+ofs].imshow(phantom2, **kw2)
meax[1+ofs].imshow(mout2[0].reshape(grid2.shape), **kw2)
meimg2 = meax[2+ofs].imshow(bout.reshape(grid2.shape), **kw2)

mecax2 = meax[2+ofs].inset_axes(bounds=[1.3, 0, 0.05, 1])
merged.colorbar(meimg2, cax=mecax2, **kwcbar2)

meimg = meax[0].imshow(image.reshape(resolution, order='F'), cmap='Blues')
mecax = meax[0].inset_axes(bounds=[1.1, 0, 0.05, 1])
merged.colorbar(meimg, cax=mecax, label='Signal [-]')

meax[0].text(32/40*resolution[0], 0.5/4*resolution[1], 'Central column', rotation='vertical')
meax[0].contour(image_column.reshape(resolution).T, levels=[0], colors='k', linewidths=0.5)
meax[0].set_title('Artificial Image')

# merged.savefig('res2m.png', bbox_inches='tight', pad_inches=0)

plt.show()

In [ ]:
gevf2 = plt.figure(figsize=(6,1.5))
gevax1, gevax2 = gevf2.subplots(1,2)

gevax1.set_xlabel('R [-]')
gevax1.set_ylabel('z [-]')
gevax1.set_title('GEV')
img = gevax1.imshow(gout[0].reshape(grid.shape), extent=grid.extent, norm=norm)

gevcax2 = gevax1.inset_axes(bounds=[1.1, 0, 0.05, 1])

gevf2.colorbar(img, cax=gevcax2, ticks=[-10, -5, 0, 50, 100], label='Emissivity [-]')

gevax2.plot(sig, '.', markersize=2, alpha=.8, color='k', label='Signal')
# ax2.errorbar(chnls, sig, yerr=errors, color='k', capsize=3, ls='', marker='+', label=signal)
gevax2.plot(gmat.dot(gout[0].flatten()), '.', markersize=2, alpha=.8, label='Retrofit')
gevax2.set_ylabel('Signal [-]')
gevax2.set_xlabel('Channel [-]')
gevax2.legend()

# gevf2.savefig('gevrf.png', bbox_inches='tight', pad_inches=0.05)

plt.show()

In [ ]:
f2 = plt.figure(figsize=(6,1.5), dpi=200)
ax1, ax2 = f2.subplots(1,2)

ax1.set_xlabel('R [-]')
ax1.set_ylabel('z [-]')
ax1.set_title('SVD')
img = ax1.imshow(sout[0].reshape(grid.shape), origin='lower', extent=grid.extent, norm=norm)

cax = ax1.inset_axes(bounds=[1.1, 0, 0.05, 1])
f2.colorbar(img, cax=cax, label='Emissivity [-]', ticks=[-10, -5, 0, 50, 100], spacing='proportional')
cax.set_ylabel('Emissivity [-]')

ax2.plot(sig, '.', markersize=2, alpha=.8, color='k', label='Signal')
# ax2.errorbar(chnls, sig, yerr=errors, color='k', capsize=3, ls='', marker='+', label=signal)
ax2.plot(gmat.dot(sout[0].flatten()), '.', markersize=2, alpha=.8, label='Retrofit')
ax2.set_ylabel('Signal [-]')
ax2.set_xlabel('Channel [-]')
ax2.legend()

# f2.savefig('svdrf.png', bbox_inches='tight', pad_inches=0.05)

plt.show()

In [ ]:
mfrf2 = plt.figure(figsize=(6,1.5))
mfrax1, mfrax2 = mfrf2.subplots(1,2)

mfrax1.set_xlabel('R [-]')
mfrax1.set_ylabel('z [-]')
mfrax1.set_title('MFR')
img = mfrax1.imshow(mout[0].reshape(grid.shape), extent=grid.extent, norm=norm)

mfrcax2 = mfrax1.inset_axes(bounds=[1.1, 0, 0.05, 1])
mfrf2.colorbar(img, cax=mfrcax2, label='Emissivity [-]', ticks=[-10, -5, 0, 50, 100], spacing='proportional')

mfrax2.plot(sig, '.', markersize=2, alpha=.8, color='k', label='Signal')
# ax2.errorbar(chnls, sig, yerr=errors, color='k', capsize=3, ls='', marker='+', label=signal)
mfrax2.plot(gmat.dot(mout[0].flatten()), '.', markersize=2, alpha=.8, label='Retrofit')
mfrax2.set_ylabel('Signal [-]')
mfrax2.set_xlabel('Channel [-]')
mfrax2.legend()

# mfrf2.savefig('mfrrf.png', bbox_inches='tight', pad_inches=0.05)

plt.show()